In [58]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import normalize
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error, mean_squared_error
from sklearn.linear_model import SGDRegressor,LinearRegression
pd.options.display.max_columns = 200
kf = KFold(n_splits=5)
kf.get_n_splits(train)

5

**Читаем датасеты**

In [59]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv("sample_submission.csv")

В Ходе сложных эмпирических испытаний были выбраны следующие значимые для моделей метрики 

**LotArea** - Площадь участка

**YearBuilt** - Год постройки

**YearRemodAdd** - Год последней ренновации


**Готовим данные**

In [67]:
#Ищем дырки
test['LotAreaLog','GarageCars'].isnull().sum()[train.isnull().sum() > 0]

KeyError: ('LotAreaLog', 'GarageCars')

In [73]:
# Немного регуляризируем

train['SalePriceLog'] = np.log1p(train['SalePrice'])

train['LotAreaLog'] = np.log1p(train['LotArea'])
test['LotAreaLog'] = np.log1p(test['LotArea'])

train['YearBuiltLog'] = np.log1p(train['YearBuilt'])
test['YearBuiltLog'] = np.log1p(test['YearBuilt'])

train['GarageAreaLog'] = np.log1p(train['GarageArea'])
test['GarageAreaLog'] = np.log1p(test['GarageArea'])

train['YearRemodAddLog'] = np.log1p(train['YearRemodAdd'])
test['YearRemodAddLog'] = np.log1p(test['YearRemodAdd'])

train['GarageCarsLog'] = np.log1p(train['GarageCars'])
test['GarageCarsLog'] = np.log1p(test['GarageCars'])

train['GrLivAreaLog'] = np.log1p(train['GrLivArea'])
test['GrLivAreaLog'] = np.log1p(test['GrLivArea'])

used_params = ['LotAreaLog','YearBuiltLog','YearRemodAddLog','GarageCars','OverallCond','OverallQual','GrLivAreaLog']

**Используем SGDRegressor**

In [74]:
model = SGDRegressor()
rmse_list = []

for i, (train_index, test_index) in enumerate(kf.split(train)):
    
    X_train, X_test = train.loc[train_index, used_params], train.loc[test_index, used_params]
    y_train, y_test = train.loc[train_index, 'SalePriceLog'], train.loc[test_index, 'SalePriceLog']

    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    rmse = np.sqrt(mean_squared_log_error(predict,y_test))
    rmse_list.append(rmse)
    print(i, rmse)
    
print("Mean rmse for 5-fold: {}".format(str(np.mean(rmse_list))))
print("std rmse for 5-fold: {}".format(str(np.std(rmse_list))))

0 0.012704372260183823
1 0.016549356744288514
2 0.017955429724076894
3 0.013155924573074658
4 0.015021737305460067
Mean rmse for 5-fold: 0.015077364121416793
std rmse for 5-fold: 0.0019887754297088773


C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


**Используем LinearRegression**

In [75]:
model = LinearRegression()
rmse_list = []

for i, (train_index, test_index) in enumerate(kf.split(train)):
    
    X_train, X_test = train.loc[train_index, used_params], train.loc[test_index, used_params]
    y_train, y_test = train.loc[train_index, 'SalePriceLog'], train.loc[test_index, 'SalePriceLog']

    model.fit(X_train, y_train)
    predict = model.predict(X_test)
    rmse = np.sqrt(mean_squared_log_error(predict,y_test))
    rmse_list.append(rmse)
    print(i, rmse)
    
print("Mean rmse for 5-fold: {}".format(str(np.mean(rmse_list))))
print("std rmse for 5-fold: {}".format(str(np.std(rmse_list))))

0 0.010843287886159347
1 0.01221111080656386
2 0.012183978886061366
3 0.010947417557020247
4 0.012811619480089846
Mean rmse for 5-fold: 0.011799482923178934
std rmse for 5-fold: 0.0007722718959947131


**Упаковываем все в ответ**

In [71]:
test=test.fillna(0)

In [77]:
model = LinearRegression()
model.fit(train[used_params], train['SalePriceLog'])

prediction = model.predict(test[used_params])
sample_submission.head()
sample_submission['SalePrice']=  np.exp( prediction)-1
sample_submission.to_csv('submission.csv', index = None)